<a href="https://colab.research.google.com/github/DimitriosSpanos/GroZi-3.2k-YOLOX-Detection/blob/main/Tech_Test_Data_Scientist_Intern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Dimitrios Spanos - Be Memorable AI***

This Notebook uses the YOLOX Architecture from "https://github.com/Megvii-BaseDetection/YOLOX"

In [ ]:
# Setup YOLOX and PyTorch
!git clone https://github.com/Megvii-BaseDetection/YOLOX
%cd YOLOX
!pip3 install -U pip && pip3 install -r requirements.txt
!pip3 install -v -e .  # or  python3 setup.py develop


#%cd /content/
#!git clone https://github.com/NVIDIA/apex
#%cd apex
#!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [2]:
# Google Drive has "grozi_coco" folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
coco_annotation_file_path = "/content/drive/MyDrive/grozi_coco/annotations/instances_val2017.json"
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from PIL import Image
import requests
from pycocotools.coco import COCO

coco_annotation = COCO(annotation_file=coco_annotation_file_path)

# Category IDs.
cat_ids = coco_annotation.getCatIds()
print(f"Number of Unique Categories: {len(cat_ids)}")
print("Category IDs:")
print(cat_ids)  # The IDs are not necessarily consecutive.

# # All categories.
cats = coco_annotation.loadCats(cat_ids)
cat_names = [cat["name"] for cat in cats]
print("Categories Names:")
print(cat_names)

# # Category ID -> Category Name.
query_id = cat_ids[0]
query_annotation = coco_annotation.loadCats([query_id])[0]
query_name = query_annotation["name"]
query_supercategory = query_annotation["supercategory"]
print("Category ID -> Category Name:")
print(
    f"Category ID: {query_id}, Category Name: {query_name}, Supercategory: {query_supercategory}"
)

# Category Name -> Category ID.
query_name = cat_names[2]
query_id = coco_annotation.getCatIds(catNms=[query_name])[0]
print("Category Name -> ID:")
print(f"Category Name: {query_name}, Category ID: {query_id}")

# # Get the ID of all the images containing the object of the category.
img_ids = coco_annotation.getImgIds(catIds=[query_id])
# print(f"Number of Images Containing {query_name}: {len(img_ids)}")

# Pick one image.
img_id = img_ids[0]
img_info = coco_annotation.loadImgs([img_id])[0]
img_file_name = img_info["file_name"]
img_url = img_info["coco_url"]
print(
    f"Image ID: {img_id}, File Name: {img_file_name}, Image URL: {img_url}"
)

# Get all the annotations for the specified image.
ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
anns = coco_annotation.loadAnns(ann_ids)
print(f"Annotations for Image ID {img_id}:")
print(anns)

# # Use URL to load image.
# im = Image.open(requests.get(img_url, stream=True).raw)

# # Save image and its labeled version.
# plt.axis("off")
# plt.imshow(np.asarray(im))
# plt.savefig(f"{img_id}.jpg", bbox_inches="tight", pad_inches=0)
# # Plot segmentation and bounding box.
# coco_annotation.showAnns(anns, draw_bbox=True)
# plt.savefig(f"{img_id}_annotated.jpg", bbox_inches="tight", pad_inches=0)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Number of Unique Categories: 185
Category IDs:
[30, 65, 86, 1006, 150, 151, 153, 154, 173, 174, 175, 176, 198, 211, 212, 233, 234, 240, 255, 1013, 262, 310, 306, 311, 331, 334, 1017, 341, 342, 343, 344, 345, 1018, 348, 1019, 353, 1020, 354, 1021, 355, 1022, 366, 415, 419, 437, 438, 439, 440, 441, 448, 449, 450, 460, 475, 476, 477, 478, 479, 480, 481, 482, 483, 1032, 484, 1033, 485, 488, 489, 499, 500, 501, 506, 507, 508, 515, 516, 517, 521, 520, 522, 523, 524, 525, 526, 527, 530, 539, 544, 545, 551, 552, 553, 554, 555, 559, 560, 561, 1037, 562, 563, 564, 582, 583, 584, 591, 593, 594, 595, 596, 603, 616, 624, 625, 1041, 627, 628, 629, 630, 1045, 684, 702, 704, 709, 710, 718, 719, 720, 730, 737, 1047, 746, 747, 748, 753, 754, 759, 40, 1217, 790, 796, 797, 804, 810, 816, 840, 841, 844, 845, 846, 872, 889, 1064, 1065, 1066, 1076, 1080, 1095, 1106, 1107, 1109, 1110, 1111, 1112, 1114, 1119, 1121, 1124, 1125, 1

KeyError: ignored

In [ ]:
%cd YOLOX/
!ln -s drive/MyDrive/grozi_coco ./datasets/grozi_coco

from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

##REPLACE this cell with your classnames stripped of whitespace and lowercase
%%writetemplate /content/YOLOX/yolox/data/datasets/coco_classes.py

COCO_CLASSES = (
  "",
)

NUM_CLASSES = 1
!sed -i -e 's/self.num_classes = 20/self.num_classes = {NUM_CLASSES}/g' "/content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py"

%cd /content/
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_s.pth
%cd /content/YOLOX/